# Trabajo Practico 2 - Grupo 04 - Backbenchers

**Integrantes:**


* Catalina Basso
* Cristobal Alvarez
* Lucas Ruiz
* Dalmiro Vilaplana  

## Lectura CSV


In [ ]:
#colocamos aqui todos los imports
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
!pip install nltk
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import ComplementNB
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.naive_bayes import BernoulliNB
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import re
import string
import joblib
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
import numpy as np
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope


In [ ]:
# Eliminar puntuación y convertir a minúsculas
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\d+", "", text)
    text = text.strip()
    return text

# Configurar stopwords
stop_words = set(stopwords.words('spanish'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

stemmer = SnowballStemmer('spanish')
def stem_text(text):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)


Primero cargamos y vemos nuestro dataset

In [ ]:
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_train.csv')
train.head()

In [ ]:
train.shape

In [ ]:
test_kaggle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tp2_test.csv')
test_kaggle.head()

In [ ]:
test_kaggle.shape

# **Bayes Naive**

## Bayes Naive 1

### Algoritmo

#### Paso 1: Preprocesamiento de datos

In [ ]:
# Para mayor simplicidad, lo pasamos a lowercase
train['review_es'] = train['review_es'].str.lower()

#### Paso 2: Separo mi modelo en train y test segun que quiero predecir

In [ ]:
X = train['review_es']
y = train['sentimiento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Paso 3: Vectorizar el dataset usando TF-IDF

- **TF (Frecuencia de Término):** Refleja la importancia de una palabra dentro de un documento específico. Palabras que aparecen más veces en un documento tendrán un valor TF más alto.
- **IDF (Frecuencia Inversa de Documento):** Reduce la importancia de las palabras comunes y aumenta la de las palabras raras. Palabras que aparecen en muchos documentos del corpus tendrán un valor IDF bajo.
- **TF-IDF: ** Proporciona un valor que refleja la importancia de una palabra en un documento específico, ajustado por la frecuencia de esa palabra en todo el corpus.

Este enfoque ayuda a capturar la relevancia de términos distintivos mientras se reduce la influencia de términos comunes que son menos informativos.

Como el modelo no distingue las palabras en español, necesitamos mostrarle cuales son los conectores en nuestro idioma

Instalo la biblioteca nltk para poder obtener una lista de palabras comunes en español que no me aportan demasiada informacion para lo que yo quiero hacer y asi poder desestimarlas.

In [ ]:
spanish_stop_words = stopwords.words('spanish')
print(spanish_stop_words)

In [ ]:
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

#### Paso 4: Entrenar con el clasificador de Bayes Naive

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

Usamos el clasificador Multinomial ya que las características de nuestro dataset son representadas por frecuencias o conteos de eventos, como en el caso de la clasificación de texto (frecuencia de palabras).

#### Paso 5: Evaluar que tan bien predice mi modelo

In [ ]:
y_pred = nb_classifier.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Precisión del modelo:", accuracy)
print("Informe de clasificación:")
print(report)

#### Paso 6: Prediccion con el modelo Test dado

In [ ]:
# Extrae los textos de la columna 'review_es' de tus nuevos datos
nuevos_textos = test_kaggle['review_es']

# Preprocesamiento de los datos nuevos
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos_tfidf = vectorizer.transform(nuevos_textos)

# Predicción utilizando el modelo entrenado
predicciones_nuevos = nb_classifier.predict(X_nuevos_tfidf)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevos

columnas_mostrar = ['ID', 'sentimiento']
resultado_final = test_kaggle[columnas_mostrar]

# Muestra el resultado final
print(resultado_final)

Me guardo la prediccion para subirla a Kaggle

In [ ]:
resultado_final[columnas_mostrar].to_csv('bayes_naive.csv', index=False)

## Bayes Naive 2

Vamos a probar modificando hiperparametros del modelo anterior para ver si mejora la prediccion

Como el Paso 1, el Paso 2 y el Paso 3 ya estan hechos, los hago "rapido" para reiniciar el algoritmo.

In [ ]:
train['review_es'] = train['review_es'].str.lower()

X = train['review_es']
y = train['sentimiento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Paso 4: Cross Validation

Defino el modelo y entreno con el clasificador de Bayes Naive. Pero ahora tambien busco los mejores hiperparametros usando CrossValidation y entreno mi modelo con los parametros hallados

In [ ]:
nb_classifier = MultinomialNB()

param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'fit_prior': [True, False]
}

grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_tfidf, y_train)

best_params = grid_search.best_params_

nb_classifier_optimized = MultinomialNB(**best_params)
nb_classifier_optimized.fit(X_train_tfidf, y_train)

### Paso 5: Evaluo mi modelo

In [ ]:
y_pred_optimized = nb_classifier_optimized.predict(X_test_tfidf)
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
report_optimized = classification_report(y_test, y_pred_optimized, target_names=['negativo', 'positivo'])

print("Precisión del modelo optimizado:", accuracy_optimized)
print("Informe de clasificación del modelo optimizado:")
print(report_optimized)

### Paso 6: Prediccion con el modelo Test dado

In [ ]:
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos_tfidf = vectorizer.transform(test_kaggle['review_es'])

# Predicción utilizando el modelo entrenado
predicciones_nuevas_optimizado = nb_classifier_optimized.predict(X_nuevos_tfidf)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevas_optimizado

# Guardar las predicciones en un archivo CSV
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_crossval = test_kaggle[columnas_mostrar]

Me guardo la prediccion para subirla a Kaggle

In [ ]:
resultado_final_crossval.to_csv('bayes_naive_cv.csv', index=False)

## Bayes Naive 3

Probemos que sucede si tratamos de utilizar F1-Score con metrica macro, y cambiando nuestro modelo de bayes a uno de complemento.

In [ ]:
train['review_es'] = train['review_es'].str.lower()

X = train['review_es']
y = train['sentimiento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Paso 4: Uso Complement

In [ ]:
clf = ComplementNB()

param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'fit_prior': [True, False],
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train_tfidf, y_train)

best_params = grid_search.best_params_

nb_complemento = ComplementNB(**best_params)
nb_complemento.fit(X_train_tfidf, y_train)

### Paso 5: Evaluo mi modelo

In [ ]:
y_pred = nb_complemento.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['negativo', 'positivo'])

print("Precisión del modelo:", accuracy)
print("Informe de clasificación del modelo:")
print(report)

### Paso 6: Prediccion con el modelo Test dado

In [ ]:
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos_tfidf = vectorizer.transform(test_kaggle['review_es'])

# Predicción utilizando el modelo entrenado
predicciones_nuevas = nb_complemento.predict(X_nuevos_tfidf)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevas

# Guardar las predicciones en un archivo CSV
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_comp = test_kaggle[columnas_mostrar]

print(resultado_final_comp)

Me guardo la prediccion para subirla a Kaggle

In [ ]:
resultado_final_comp.to_csv('bayes_naive_comp.csv', index=False)

## Bayes Naive 4

Vamos a utilizar un proceso llamado Lemmatization para transformar las palabras a su version 'basica' o una que podemos encontrar en el diccionario, para ver si nuestro modelo mejora.

### Paso 1: Lematizamos los datos de entrenamiento primero

In [26]:
X = train['review_es']
y = train['sentimiento']

# Lematizamos los datos de entrenamiento
lemmatized_X = []
lemmatizer = WordNetLemmatizer()
for text in X:
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = " ".join(lemmatized_words)
    lemmatized_X.append(lemmatized_text)


### Paso 2: hacemos el split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(lemmatized_X, y, test_size=0.3, random_state=2)

vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Paso 3: entrenamos el modelo, buscando los mejores hiperparametros

In [28]:
clf = ComplementNB()

param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'fit_prior': [True, False],
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train_tfidf, y_train)

best_params = grid_search.best_params_

nb_complemento = ComplementNB(**best_params)
nb_complemento.fit(X_train_tfidf, y_train)

ComplementNB(alpha=1)

In [29]:
predictions = nb_complemento.predict(X_test_tfidf)

### Paso 4: Evaluo mi modelo

In [30]:
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions, target_names=['negativo', 'positivo'])

print("Precisión del modelo:", accuracy)
print("Informe de clasificación del modelo:")
print(report)

Precisión del modelo: 0.8596666666666667
Informe de clasificación del modelo:
              precision    recall  f1-score   support

    negativo       0.86      0.86      0.86      7499
    positivo       0.86      0.86      0.86      7501

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



### Paso 5: Prediccion para el modelo de test dado

In [31]:
X = test_kaggle['review_es']
y = test_kaggle['sentimiento']

# Lematizamos los datos de entrenamiento
lemmatized_X = []
lemmatizer = WordNetLemmatizer()
for text in X:
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = " ".join(lemmatized_words)
    lemmatized_X.append(lemmatized_text)

test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos_tfidf = vectorizer.transform(lemmatized_X)

# Predicción utilizando el modelo entrenado
predicciones_nuevas = nb_complemento.predict(X_nuevos_tfidf)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevas

# Guardar las predicciones en un archivo CSV
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_lemmantizado = test_kaggle[columnas_mostrar]

print(resultado_final_lemmantizado)

         ID sentimiento
0     60000    negativo
1     60001    negativo
2     60002    negativo
3     60003    positivo
4     60004    negativo
...     ...         ...
8594  68594    positivo
8595  68595    negativo
8596  68596    positivo
8597  68597    negativo
8598  68598    negativo

[8599 rows x 2 columns]


Me guardo la prediccion para subirla a Kaggle

In [32]:
resultado_final_lemmantizado.to_csv('bayes_naive_cv_lematizado.csv', index=False)

## Bayes Naive 5


In [33]:
train['review_es'] = train['review_es'].str.lower()

X = train['review_es']
y = train['sentimiento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Paso 4: Uso Bernoulli


In [34]:
Bernoulli = BernoulliNB()
Bernoulli.fit(X_train_tfidf, y_train)

BernoulliNB()

In [35]:
y_pred = Bernoulli.predict(X_test_tfidf)

### Paso 5: Evaluo mi modelo

In [36]:
classification_rep = classification_report(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
print('Classification Report with BernoulliNB:')
print(classification_rep)

Accuracy: 0.8457
Classification Report with BernoulliNB:
              precision    recall  f1-score   support

    negativo       0.82      0.88      0.85      4961
    positivo       0.87      0.82      0.84      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



### Paso 6: Prediccion usando el modelo de test dado

In [37]:
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()
X_nuevos_tfidf = vectorizer.transform(test_kaggle['review_es'])
predicciones_nuevas = Bernoulli.predict(X_nuevos_tfidf)
test_kaggle['sentimiento'] = predicciones_nuevas_optimizado

# Guardar las predicciones en un archivo CSV
columnas_mostrar = ['ID', 'sentimiento']
resultado_final_ber = test_kaggle[columnas_mostrar]


In [38]:
resultado_final_ber.to_csv('bayes_naive_Bernoulli.csv', index=False)

##Bayes Naive 6

In [39]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)

In [40]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [42]:
bernoulli2 = BernoulliNB()

In [43]:
param_grid = {
    'alpha': [0.01, 0.1, 1, 10],
    'binarize': [0.0, 0.5, 1.0]
}

### Paso 4: Cross Validation

In [44]:
grid_search = GridSearchCV(estimator=bernoulli2, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BernoulliNB(),
             param_grid={'alpha': [0.01, 0.1, 1, 10],
                         'binarize': [0.0, 0.5, 1.0]},
             scoring='f1_macro')

In [45]:
best_params = grid_search.best_params_
best_bernoulli = grid_search.best_estimator_

### Paso 5: Evaluo mi modelo

In [46]:
y_pred = best_bernoulli.predict(X_test)
print("Mejores hiperparámetros:", best_params)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Mejores hiperparámetros: {'alpha': 1, 'binarize': 0.0}
Accuracy en el conjunto de prueba: 0.8213
Reporte de clasificación:
               precision    recall  f1-score   support

    negativo       0.82      0.83      0.82      4961
    positivo       0.83      0.82      0.82      5039

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000



### Paso 6: Prediccion para el modelo de test dado

In [47]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(stem_text)
X_test_kaggle = vectorizer.transform(test_kaggle['review_clean']).toarray()

In [48]:
y_pred_kaggle = best_bernoulli.predict(X_test_kaggle)

In [49]:
test_kaggle

ID                                          review_es sentimiento  \
0     60000  la mayor virtud de esta película es su existen...    negativo   
1     60001  no soy un experto cinéfilo, pero pocas veces m...    negativo   
2     60002  si no eres un incondicional del humor estilo t...    negativo   
3     60003  no sé qué está pasando, si la gente se deja ll...    positivo   
4     60004  pero cuando amanece,y me quedo solo,siento en ...    negativo   
...     ...                                                ...         ...   
8594  68594  buena no, lo siguiente. por fin un film serio ...    positivo   
8595  68595  me esperaba mucho, pero que mucho, más.guión m...    negativo   
8596  68596  de mal cuerpo como sensación al finalizar, de ...    positivo   
8597  68597  los que han añadido comentarios os lo han dich...    negativo   
8598  68598  fui a ver esta película de cine con entusiasmo...    positivo   

                                           review_clean  
0     mayor virtud pelicul existenciael hech pod jug...  
1     expert cinefil poc vec tan jueg sal cin autent...  
2     si incondicional humor estil tel si termin cap...  
3     se pas si gent dej llev mod si verd gust sol v...  
4     amanecey qued solosient fondoun mar vacioun se...  
...                                                 ...  
8594  buen siguient fin film seri interpret estel fo...  
8595  esper masguion vist noved absolut previs comie...  
8596  mal cuerp sensacion finaliz histori interes te...  
8597  añad comentari dich clar guion pesim actuacion...  
8598  ver pelicul cin entusiasm promet tal vez tard ...  

[8599 rows x 4 columns]

In [50]:
submission = pd.DataFrame({'id': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})

In [51]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/submission_bernoulli_v2.csv', index=False)

## Bayes Naive 7: Bernoulli con mejores parametros

In [52]:
# Limpiar, eliminar stopwords y lematizar texto de entrenamiento
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)

# Vectorizar el texto
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Paso 4: Cross Validation

In [53]:
# Modelo de Bernoulli Naive Bayes
bernoulli3 = BernoulliNB()

# Definir hiperparametros
param_grid = {
    'alpha': [0.01, 0.1, 1, 10],
    'binarize': [0.0, 0.5, 1.0]
}

# Realizar Grid-Search
grid_search = GridSearchCV(estimator=bernoulli3, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# mejores hiperparámetros y el mejor estimador
best_params = grid_search.best_params_
best_bernoulli = grid_search.best_estimator_

### Paso 5: Evaluo mi modelo

In [54]:
# Predecir en el conjunto de prueba
y_pred = best_bernoulli.predict(X_test)
print("Mejores hiperparámetros:", best_params)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))


Mejores hiperparámetros: {'alpha': 0.1, 'binarize': 0.0}
Accuracy en el conjunto de prueba: 0.8278
Reporte de clasificación:
               precision    recall  f1-score   support

    negativo       0.84      0.81      0.82      4961
    positivo       0.82      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Paso 6: Prediccion para el modelo de test dado

In [55]:
# Limpiar, eliminar stopwords y lematizar texto de prueba
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)
X_test_kaggle = vectorizer.transform(test_kaggle['review_clean']).toarray()

# Predecir en el conjunto de datos de Kaggle
y_pred_kaggle = best_bernoulli.predict(X_test_kaggle)

# Crear el archivo de envío
submission = pd.DataFrame({'id': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('Bayes_Naive_Bernoulli_v3.csv', index=False)

## Bayes Naive 8: tratando de combinar mis mejores modelos

In [56]:
# Limpiar, eliminar stopwords, lematizar y stemmear texto de entrenamiento
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text).apply(stem_text)

# Vectorizar el texto
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo de Bernoulli Naive Bayes
bernoulli4 = BernoulliNB()

#hiperparametros
param_grid = {
    'alpha': [0.01, 0.1, 1, 10],
    'binarize': [0.0, 0.5, 1.0]
}

#Grid-Search
grid_search = GridSearchCV(estimator=bernoulli4, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# mejores hiperparámetros y el mejor estimador
best_params = grid_search.best_params_
best_bernoulli = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_bernoulli.predict(X_test)
print("Mejores hiperparámetros:", best_params)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Mejores hiperparámetros: {'alpha': 0.1, 'binarize': 0.0}
Accuracy en el conjunto de prueba: 0.8279
Reporte de clasificación:
               precision    recall  f1-score   support

    negativo       0.84      0.81      0.82      4961
    positivo       0.82      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [57]:
# Limpiar, eliminar stopwords y lematizar texto de prueba
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)
X_test_kaggle = vectorizer.transform(test_kaggle['review_clean']).toarray()

# Predecir en el conjunto de datos de Kaggle
y_pred_kaggle = best_bernoulli.predict(X_test_kaggle)

# Crear el archivo de envío
submission = pd.DataFrame({'id': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('Bayes_Naive_Bernoulli_v4.csv', index=False)

## Bayes Naive 9: SVM

In [58]:
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(train['review_es'], train['sentimiento'], test_size=0.2, random_state=42)

### Paso 4: Cross Validation

In [ ]:
# Crear una pipeline para integrar el vectorizador y el clasificador
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
    ('classifier', LinearSVC())
])

# hiperparametros
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100]
}

# Grid-Search
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y el mejor estimador
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Paso 5: Evaluo mi modelo

In [ ]:
# Predecir en el conjunto de prueba
y_pred = best_model.predict(X_test)
print("Mejores hiperparámetros:", best_params)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))


Mejores hiperparámetros: {'classifier__C': 0.1}
Accuracy en el conjunto de prueba: 0.8789
Reporte de clasificación:
               precision    recall  f1-score   support

    negativo       0.89      0.87      0.88      4961
    positivo       0.87      0.89      0.88      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



### Paso 6: Prediccion para el modelo de test dado

In [ ]:
# Aplicar el mismo preprocesamiento al conjunto de test
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)
y_pred_kaggle = best_model.predict(test_kaggle['review_clean'])

# Crear el archivo de envío
submission = pd.DataFrame({'id': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('svm_linear.csv', index=False)

## Bayes Naive 10

In [ ]:
# Limpiar, eliminar stopwords y lematizar texto de entrenamiento
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)

# Vectorizar el texto con bigramas y ajustar el número de características
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Ajusta max_features si es necesario
X = vectorizer.fit_transform(train['review_clean'])
y = train['sentimiento']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Paso 4: Cross Validation

In [ ]:
# Modelo de Bernoulli Naive Bayes
bernoulli5 = BernoulliNB()

#hiperparametros
param_grid = {
    'alpha': [0.01, 0.1, 1, 10],
    'binarize': [0.0, 0.5, 1.0]
}

#Grid-Search
grid_search = GridSearchCV(estimator=bernoulli5, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y el mejor estimador
best_params = grid_search.best_params_
best_bernoulli = grid_search.best_estimator_

### Paso 5: Evaluo mi modelo

In [ ]:
# Predecir en el conjunto de prueba
y_pred = best_bernoulli.predict(X_test)
print("Mejores hiperparámetros:", best_params)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Mejores hiperparámetros: {'alpha': 0.1, 'binarize': 0.0}
Accuracy en el conjunto de prueba: 0.8278
Reporte de clasificación:
               precision    recall  f1-score   support

    negativo       0.84      0.81      0.82      4961
    positivo       0.82      0.85      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



### Paso 6: Prediccion para el modelo de test dado

In [ ]:
# Limpiar, eliminar stopwords y lematizar texto de prueba
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)
X_test_kaggle = vectorizer.transform(test_kaggle['review_clean'])

# Predecir en el conjunto de datos de Kaggle
y_pred_kaggle = best_bernoulli.predict(X_test_kaggle)

# Crear el archivo de envío
submission = pd.DataFrame({'id': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('Bayes_Naive_Bernoulli_v5.csv', index=False)

## Bayes Naive 11

In [ ]:
# Limpiar, eliminar stopwords y lematizar texto de entrenamiento
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)

In [ ]:
# Vectorizar el texto con bigramas y ajustar el número de características
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), max_df=0.75, min_df=5)  # Ajustar parámetros
X = vectorizer.fit_transform(train['review_clean'])
y = train['sentimiento']

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Paso 4: Cross Validation

In [ ]:
multinomial_nb = MultinomialNB()

#hiperparametros
param_grid = {
    'alpha': [0.01, 0.1, 1, 10]
}

#Grid-Search
grid_search = GridSearchCV(estimator=multinomial_nb, param_grid=param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y el mejor estimador
best_params = grid_search.best_params_
best_multinomial = grid_search.best_estimator_

In [ ]:
# Cross-Validation-Score
scores = cross_val_score(best_multinomial, X, y, cv=5, scoring='accuracy')
print("Accuracy promedio de validación cruzada:", scores.mean())

### Paso 5: Evaluo mi modelo

In [ ]:
# Predecir en el conjunto de prueba
y_pred = best_multinomial.predict(X_test)
print("Mejores hiperparámetros:", best_params)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

### Paso 6: Prediccion para el modelo de test dado

In [ ]:
# Limpiar, eliminar stopwords y lematizar texto de prueba
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)
X_test_kaggle = vectorizer.transform(test_kaggle['review_clean'])

# Predecir en el conjunto de datos de Kaggle
y_pred_kaggle = best_multinomial.predict(X_test_kaggle)

# Crear el archivo de envío
submission = pd.DataFrame({'id': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('Multinomial_Naive_Bayes_Optimizado.csv', index=False)

## Voting

In [ ]:
# Limpiar, eliminar stopwords y lematizar texto de entrenamiento
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)

In [ ]:
# Vectorizar el texto con bigramas y ajustar el número de características
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), max_df=0.75, min_df=5)  # Ajustar parámetros
X = vectorizer.fit_transform(train['review_clean'])
y = train['sentimiento']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Paso 4: Cross Validation

In [ ]:
param_grid_lr = {'C': [0.01, 0.1, 1, 10, 100]}

# Grid-Search para Logistic Regression
grid_search_lr = GridSearchCV(estimator=LogisticRegression(max_iter=200), param_grid=param_grid_lr, cv=5, scoring='f1_macro')
grid_search_lr.fit(X_train, y_train)

# Obtener los mejores hiperparámetros para Logistic Regression
best_params_lr = grid_search_lr.best_params_
best_log_reg = grid_search_lr.best_estimator_

In [ ]:
# Entrenar modelos base
model_lr = LogisticRegression(max_iter=200, C=best_params_lr['C']).fit(X_train, y_train)
model_linear_svc = LinearSVC().fit(X_train, y_train)
model_rf = RandomForestClassifier(n_estimators=50, max_depth=10, max_features='sqrt', random_state=42).fit(X_train, y_train)

In [ ]:
# Crear el Voting Classifier con voting='hard'
voting_clf = VotingClassifier(estimators=[
    ('lr', model_lr),
    ('svc', model_linear_svc),
    ('rf', model_rf)
], voting='hard')

# Entrenar el Voting Classifier
voting_clf.fit(X_train, y_train)

### Paso 5: Evaluo mi modelo

In [ ]:
# Predecir en el conjunto de prueba
y_pred = voting_clf.predict(X_test)
print("Accuracy del Voting Classifier en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

### Paso 6: Prediccion para el modelo de test dado

In [ ]:
# Limpiar, eliminar stopwords y lematizar texto de prueba
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)

# Vectorizar el texto de prueba
X_test_kaggle = vectorizer.transform(test_kaggle['review_clean'])

# Predecir en el conjunto de datos de Kaggle
y_pred_kaggle = voting_clf.predict(X_test_kaggle)

# Crear el archivo de envío
submission = pd.DataFrame({'id': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('Voting_Classifier_2.csv', index=False)

## Bayes Naive 12

Vamos a intentar hacer un modelo que combine las caracteristicas que dieron mejores resultados previamente. Para eso, haremos lo siguiente:

#### Paso 3: Vectorizar el dataset usando TF-IDF

Utilizamos TF-ID como antes pero utilizando bigramas y aumentando la cantidad maxima de features

In [ ]:
X = train['review_es']
y = train['sentimiento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer(stop_words=stopwords.words('spanish'), ngram_range=(1, 2), max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

#### Paso 4: Entrenar con el clasificador de Bayes Naive

Optimizamos los hiperparaemtros con cross validation y bernoulli, ampliando la búsqueda de alpha y binarize para encontrar mejores combinaciones.

In [ ]:
bernoulli_nb = BernoulliNB()
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 50],
    'binarize': [0.0, 0.1, 0.5, 1.0]
}
grid_search = GridSearchCV(estimator=bernoulli_nb, param_grid=param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)
best_params = grid_search.best_params_
best_bernoulli = grid_search.best_estimator_

#### Paso 5: Evaluar que tan bien predice mi modelo

In [ ]:
y_pred = best_bernoulli.predict(X_test_tfidf)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Informe de clasificación:\n", classification_report(y_test, y_pred))

### Paso 6: Prediccion con el modelo Test dado

In [ ]:
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords).apply(lemmatize_text)
X_test_kaggle = vectorizer.transform(test_kaggle['review_clean'])

y_pred_kaggle = best_bernoulli.predict(X_test_kaggle)

Me guardo la prediccion para subirla a Kaggle

In [ ]:
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Bayes_Naive_BernoulliV3.5.csv', index=False)

## Hyperopt

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train['review_es'], train['sentimiento'], test_size=0.2, random_state=42)

X_train_clean = X_train.apply(clean_text).apply(remove_stopwords)
X_test_clean = X_test.apply(clean_text).apply(remove_stopwords)

In [ ]:
# Espacio de búsqueda para Hyperopt
space = hp.choice('classifier_type', [
    {
        'type': 'BernoulliNB',
        'alpha': hp.loguniform('alpha_nb', np.log(0.01), np.log(50)),
        'binarize': hp.uniform('binarize_nb', 0.0, 1.0)
    },
    {
        'type': 'LogisticRegression',
        'C': hp.loguniform('C_lr', np.log(0.01), np.log(50))
    },
    {
        'type': 'RandomForestClassifier',
        'n_estimators': scope.int(hp.quniform('n_estimators_rf', 100, 500, 100))
    }
])

In [ ]:
# Función de objetivo para Hyperopt
def objective(params):
    if params['type'] == 'BernoulliNB':
        clf = BernoulliNB(alpha=params['alpha'], binarize=params['binarize'])
    elif params['type'] == 'LogisticRegression':
        clf = LogisticRegression(C=params['C'], max_iter=200)
    else:
        clf = RandomForestClassifier(n_estimators=params['n_estimators'])

    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words=stopwords.words('spanish'), max_features=10000, ngram_range=(1, 2))),
        ('clf', clf)
    ])

    pipeline.fit(X_train_clean, y_train)
    y_pred = pipeline.predict(X_test_clean)
    accuracy = accuracy_score(y_test, y_pred)

    return {'loss': -accuracy, 'status': STATUS_OK}

In [ ]:
# Ejecutar la optimización bayesiana con Hyperopt
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

In [ ]:
# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", best)

In [ ]:
# Evaluar el mejor modelo encontrado por Hyperopt
if best['classifier_type'] == 0:
    best_clf = BernoulliNB(alpha=best['alpha_nb'], binarize=best['binarize_nb'])
elif best['classifier_type'] == 1:
    best_clf = LogisticRegression(C=best['C_lr'], max_iter=200)
else:
    best_clf = RandomForestClassifier(n_estimators=best['n_estimators_rf'])

best_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords.words('spanish'), max_features=10000, ngram_range=(1, 2))),
    ('clf', best_clf)
])

best_pipeline.fit(X_train_clean, y_train)
y_pred = best_pipeline.predict(X_test_clean)
print("Accuracy en el conjunto de prueba:", accuracy_score(y_test, y_pred))
print("Informe de clasificación:\n", classification_report(y_test, y_pred))

In [ ]:
# Predicción con el mejor modelo para el conjunto de test de Kaggle
test_kaggle['review_clean'] = test_kaggle['review_es'].apply(clean_text).apply(remove_stopwords)

# Hacer predicciones utilizando la mejor pipeline
y_pred_kaggle = best_pipeline.predict(test_kaggle['review_clean'])

# Crear el dataframe de predicciones para Kaggle
submission = pd.DataFrame({'ID': test_kaggle['ID'], 'sentimiento': y_pred_kaggle})

# Guardar las predicciones en un archivo CSV
submission.to_csv('Best_Pipeline_Predictions.csv', index=False)

##Bayes Naive 13

Intentamos maximizar nuestro bag of words a lo mas alto posible

### Paso 4: Cross Validation

In [ ]:
# Aplicar la limpieza y eliminación de palabras vacías
train['review_clean'] = train['review_es'].apply(clean_text).apply(remove_stopwords)

In [ ]:
train['review_clean'] = train['review_clean'].apply(stem_text)

In [ ]:
vectorizer = CountVectorizer(max_features=10000)
X = vectorizer.fit_transform(train['review_clean']).toarray()
y = train['sentimiento']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

### Paso 5: Evaluo mi modelo

In [ ]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Informe de clasificación:")
print(report)

### Prediccion con el modelo Test dado

In [ ]:
# Extrae los textos de la columna 'review_es' de tus nuevos datos
nuevos_textos = test_kaggle['review_es']

# Preprocesamiento de los datos nuevos
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos = vectorizer.transform(nuevos_textos)

# Predicción utilizando el modelo entrenado
predicciones_nuevos = nb_classifier.predict(X_nuevos)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevos

columnas_mostrar = ['ID', 'sentimiento']
resultado_final = test_kaggle[columnas_mostrar]

# Muestra el resultado final
print(resultado_final)

Me guardo la prediccion para subirla a Kaggle

In [ ]:
resultado_final[columnas_mostrar].to_csv('bayes_naive_bag_of_words_1.csv', index=False)

##Bayes Naive 14

Probamos tomar palabras de a pares en vez de unicas

### Paso 4: Cross Validation

In [ ]:
X = train['review_es']
y = train['sentimiento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [ ]:
# usamos rango para contar palabras de a 2
vectorizer = CountVectorizer(ngram_range=(1, 2))

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

### Paso 5: Evaluo mi modelo

In [ ]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Informe de clasificación:")
print(report)

### Paso 6: Prediccion con el modelo Test dado

In [ ]:
# Extrae los textos de la columna 'review_es' de tus nuevos datos
nuevos_textos = test_kaggle['review_es']

# Preprocesamiento de los datos nuevos
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos = vectorizer.transform(nuevos_textos)

# Predicción utilizando el modelo entrenado
predicciones_nuevos = nb_classifier.predict(X_nuevos)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevos

columnas_mostrar = ['ID', 'sentimiento']
resultado_final = test_kaggle[columnas_mostrar]

# Muestra el resultado final
print(resultado_final)

Me guardo la prediccion para subirla a Kaggle

In [ ]:
resultado_final[columnas_mostrar].to_csv('bayes_naive_vector_tomando_varias_palabras.csv', index=False)

##Bayes Naive 15

Probamos tomar palabras de a 2 y de a 3

### Paso 4: Cross Validation

In [ ]:
X = train['review_es']
y = train['sentimiento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [ ]:
# usamos rango para contar palabras de a 2
vectorizer = CountVectorizer(ngram_range=(1, 3))

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

### Paso 5: Evaluo mi modelo

In [ ]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Informe de clasificación:")
print(report)

### Paso 6: Prediccion con el modelo Test dado

In [ ]:
# Extrae los textos de la columna 'review_es' de tus nuevos datos
nuevos_textos = test_kaggle['review_es']

# Preprocesamiento de los datos nuevos
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos = vectorizer.transform(nuevos_textos)

# Predicción utilizando el modelo entrenado
predicciones_nuevos = nb_classifier.predict(X_nuevos)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevos

columnas_mostrar = ['ID', 'sentimiento']
resultado_final = test_kaggle[columnas_mostrar]

# Muestra el resultado final
print(resultado_final)

Me guardo la prediccion para subirla a Kaggle

In [ ]:
resultado_final[columnas_mostrar].to_csv('bayes_naive_vector_tomando_varias_palabras_2.csv', index=False)

##Bayes Naive 16

Probamos tomar palabras de a 1, 2, 3, y 4 en vez de unicas

### Paso 4: Cross Validation

In [ ]:
X = train['review_es']
y = train['sentimiento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [ ]:
# usamos rango para contar palabras de a 1,2,3 y 4

vectorizer = CountVectorizer(ngram_range=(1, 4))
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

### Paso 5: Evaluo mi modelo

In [ ]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Informe de clasificación:")
print(report)

### Paso 6: Prediccion con el modelo Test dado

In [ ]:
# Extrae los textos de la columna 'review_es' de tus nuevos datos
nuevos_textos = test_kaggle['review_es']

# Preprocesamiento de los datos nuevos
test_kaggle['review_es'] = test_kaggle['review_es'].str.lower()

# Vectorización de los datos nuevos utilizando el mismo vectorizador
X_nuevos = vectorizer.transform(nuevos_textos)

# Predicción utilizando el modelo entrenado
predicciones_nuevos = nb_classifier.predict(X_nuevos)

# Agrega las predicciones al dataframe
test_kaggle['sentimiento'] = predicciones_nuevos

columnas_mostrar = ['ID', 'sentimiento']
resultado_final = test_kaggle[columnas_mostrar]

# Muestra el resultado final
print(resultado_final)

Me guardo la prediccion para subirla a Kaggle

In [ ]:
resultado_final[columnas_mostrar].to_csv('bayes_naive_vector_tomando_varias_palabras_3.csv', index=False)